# Definitions

In [ ]:
import time 
from ml2sql import ML2SQL

# PostgreSQL
import psycopg2 as pg
con = pg.connect(CONNECTIONSTRING)
backend = "postgres"

In [ ]:
def run_query(query, con, should_print = True):
    cursor = con.cursor()
    cursor.execute(query)
    rs = cursor.fetchall()
    if not rs:
        print("Query result is empty")
    colnames = [desc[0] for desc in cursor.description]
    if should_print:
        print(colnames)
        for res in rs:
            print(res)
    cursor.close()

def run_update_query(query, con):
    cursor = con.cursor()
    cursor.execute(query)
    con.commit()
    cursor.close()

# Example model

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(4,),activation='linear', bias_initializer=tf.keras.initializers.RandomNormal()),
    tf.keras.layers.Dense(8, input_shape=(4,),activation='relu', bias_initializer=tf.keras.initializers.RandomNormal()),
    tf.keras.layers.Dense(2, input_shape=(4,),activation='sigmoid', bias_initializer=tf.keras.initializers.RandomNormal()),
    tf.keras.layers.Dense(1, activation='linear', bias_initializer=tf.keras.initializers.RandomNormal())
])

model.compile(loss='categorical_crossentropy')

# Initialize ml2sql

In [ ]:
translator = ML2SQL(con, backend, model)

# Model import

In [ ]:
model_table_name = "iris_model"
start_time = time.time()
queries = translator.model_to_relation(model_table_name)
for q in queries:
    run_update_query(q, con)
print("--- %s seconds ---" % (time.time() - start_time))
q = f"select * from {model_table_name}"
#run_query(q, con)

# Model join

In [ ]:
# Table has to exist in database
tablename = "iris"
id_col_name = "id"
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
output_col_name = "prediction"

# Build MJ query
input_query = f"Select * from {tablename}"
mj_query = translator.model_join_query(input_query, id_col_name, col_names, model_table_name, output_col_name)
# Run MJ
start_time = time.time()
run_query(mj_query, con, False)
print("--- %s seconds ---" % (time.time() - start_time))

# Verification

In [ ]:
import numpy as np

cols = ','.join(col_names)
query = f"Select id, {cols}, {output_col_name} from ({mj_query}) as t order by id"

cursor = con.cursor()
cursor.execute(query)
rs = cursor.fetchall()

arr = np.array(rs, dtype='float32')[:,1:] # drop the id col
splitted = np.hsplit(arr, [len(col_names)])
inputs = splitted[0]

# Do inference with TensorFlow
start_time = time.time()
tfpred = model.predict(inputs)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Float precision and representation is different, so convert to ints
# TODO: categorical values
#predictions = (splitted[1] * 1000000).astype(int)
#model_out = (model.predict(inputs) * 1000000).astype(int)
decimals = 5
predictions = splitted[1].round(decimals)
model_out = tfpred.round(decimals)

if not np.array_equal(predictions, model_out):
	print("Results not verified!")
	#print(np.equal(predictions, model_out))

	for idx, el in np.ndenumerate(predictions):
		if np.not_equal(el, model_out[idx]) and el != model_out[idx]:
			print(idx[0], el, model_out[idx])
else:
	print("Results verified")

In [ ]:
con.close()